In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
heart = pd.read_csv('heartd.csv')
# heart.info() 
# thal = 301 and ca = 299 where others = 303

In [2]:
# heart.describe()

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
# heart.hist(bins = 50 , figsize=(20,15))

In [4]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(heart, test_size=0.2, random_state=42)

In [5]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index,test_index in split.split(heart,heart['sex']): # we need to select some categorcal value 
    strat_train_set = heart.loc[train_index]                   # on which the output is deeply affected 
    strat_test_set = heart.loc[test_index]

In [6]:
heart = strat_train_set.copy() # copy of strat train in heart(original data)

## Looking for co-relation

In [7]:
# corr_matrix = heart.corr()
# corr_matrix['num'].sort_values(ascending = False)

In [8]:
# from pandas.plotting import scatter_matrix
# attributes = ['num', 'ca', 'oldpeak','age','thalach']
# scatter_matrix(heart[attributes],figsize = (12,8))

In [9]:
# heart.plot(kind = "scatter", x= "age",y = "num", alpha = 0.9)

In [10]:
# median = heart['thal'].median() #option 3
# heart['thal'].fillna(median)
# median1 = heart['ca'].median()
# heart['ca'].fillna(median1)

In [11]:
heart = strat_train_set.drop('num',axis=1) 
heart_labels = strat_train_set['num'].copy()


In [12]:
heart.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
count,242.000000,242.000000,242.000000,242.000000,242.000000,242.00000,242.000000,242.000000,242.000000,242.000000,242.000000,238.000000,240.000000
mean,53.768595,0.681818,3.185950,130.004132,246.838843,0.14876,0.950413,149.739669,0.334711,1.007025,1.599174,0.672269,4.700000
std,9.076220,0.466736,0.943711,16.381980,54.124672,0.35659,0.992513,22.890194,0.472867,1.142499,0.618256,0.955634,1.943124
min,29.000000,0.000000,1.000000,94.000000,126.000000,0.00000,0.000000,71.000000,0.000000,0.000000,1.000000,0.000000,3.000000
25%,46.250000,0.000000,3.000000,120.000000,211.000000,0.00000,0.000000,136.000000,0.000000,0.000000,1.000000,0.000000,3.000000
50%,55.000000,1.000000,3.000000,130.000000,239.500000,0.00000,0.000000,153.500000,0.000000,0.800000,2.000000,0.000000,3.000000
75%,60.000000,1.000000,4.000000,140.000000,275.000000,0.00000,2.000000,165.000000,1.000000,1.600000,2.000000,1.000000,7.000000
max,77.000000,1.000000,4.000000,180.000000,564.000000,1.00000,2.000000,202.000000,1.000000,6.200000,3.000000,3.000000,7.000000


In [13]:
 from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(strategy = "median")
# imputer.fit(heart)

In [14]:
# imputer.statistics_

In [15]:
# X = imputer.transform(heart) # X is a numpy array 
# heart_transf = pd.DataFrame(X, columns = heart.columns)# heart_transf --> transform data set after filling the missing values
# heart_transf.describe()  

## Creating pipelines

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
my_pipeline = Pipeline([   # pipeline takes a series of list in it
    ('imputer',SimpleImputer(strategy="median")),  
    ('std_scaler', StandardScaler()),
])

In [17]:
heart_tr = my_pipeline.fit_transform(heart) # heart_tr is a numpy array

In [18]:
heart_tr.shape

(242, 13)

## Selecting a desired model

In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

#model = LinearRegression()
#model = DecisionTreeRegressor()
model = RandomForestRegressor()
model.fit(heart_tr,heart_labels)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [20]:
some_data = heart.iloc[:5]
some_labels = heart_labels.iloc[:5]

In [21]:
prepared_data = my_pipeline.transform(some_data)  # transforming the new data to the existing data


In [22]:
model.predict(prepared_data)

array([0.05, 2.67, 0.  , 3.53, 0.35])

In [23]:
list(some_labels)

[0, 3, 0, 4, 0]

## Evaluating the Model

In [24]:
# from sklearn.metrics import mean_squared_error
# heart_predictions = model.predict(heart_tr)
# mse = mean_squared_error(heart_labels, heart_predictions)
# rmse = np.sqrt(mse)

## Using better evaluation Technique - Cross Validation

In [25]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, heart_tr,heart_labels,scoring = "neg_mean_squared_error", cv=10 )
rmse_scores = np.sqrt(-scores)


In [26]:
rmse_scores

array([0.70617562, 1.28254123, 1.0154412 , 0.89208324, 0.80156098,
       0.84613829, 0.59916956, 0.87901934, 0.95811882, 0.77670511])

In [27]:
def print_scores(scores) : 
    print("scores are : ",scores)
    print("Mean :", scores.mean())
    print("Standard Deviation:  ", scores.std())

In [28]:
print_scores(rmse_scores)

scores are :  [0.70617562 1.28254123 1.0154412  0.89208324 0.80156098 0.84613829
 0.59916956 0.87901934 0.95811882 0.77670511]
Mean : 0.8756953378185296
Standard Deviation:   0.17705867762663094


In [29]:
from joblib import dump,load
dump(model , 'Heart.joblib')


['Heart.joblib']

## Testing

In [30]:
X_test = strat_test_set.drop("num",axis=1)
Y_test = strat_test_set["num"].copy()
X_test_prepared = my_pipeline.transform(X_test)
final_predictions = model.predict(X_test_prepared)
final_mse = mean_squared_error(Y_test,final_predictions)
final_rmse = np.sqrt(final_mse)

In [31]:
print(final_predictions,list(Y_test))


[0.76 0.25 0.6  2.7  0.07 1.92 1.53 0.66 2.95 0.02 2.91 1.23 0.24 0.12
 0.03 0.93 0.54 2.01 2.05 3.23 1.97 1.05 0.49 0.27 1.35 0.01 0.39 0.48
 0.11 0.07 0.34 1.73 1.58 1.42 0.46 0.71 0.51 1.57 2.23 0.03 0.4  3.
 2.67 0.65 0.03 0.2  0.23 0.03 0.51 2.89 1.81 2.91 0.1  0.92 0.53 0.43
 0.53 0.16 0.75 2.22 0.47] [0, 0, 0, 3, 0, 3, 2, 0, 2, 0, 3, 0, 0, 0, 0, 3, 0, 1, 0, 3, 3, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 4, 2, 0, 1, 0, 0, 4, 1, 0, 0, 2, 1, 0, 0, 1, 0, 0, 0, 2, 4, 2, 0, 1, 0, 3, 1, 0, 1, 3, 1]


## Using The Model

In [32]:
from joblib import dump,load
model = load('Heart.joblib')

In [33]:
# prepared_data[0]

In [34]:
feature = np.array([[-1.18892157,  0.68313005,  0.86439266,  1.22313113,  0.00298369,
       -0.41803981, -0.9595669 ,  0.93072139, -0.70929937,  0.43238294,
       -0.97114425, -0.69625788, -0.87031794]])
model.predict(feature)

array([0.05])